<a href="https://colab.research.google.com/github/Alex-Witt/DS-Unit-1-Sprint-4-Statistical-Tests-and-Experiments/blob/master/DS_Unit_1_Sprint_Challenge_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Unit 1 Sprint Challenge 4

## Exploring Data, Testing Hypotheses

In this sprint challenge you will look at a dataset of people being approved or rejected for credit.

https://archive.ics.uci.edu/ml/datasets/Credit+Approval

Data Set Information: This file concerns credit card applications. All attribute names and values have been changed to meaningless symbols to protect confidentiality of the data. This dataset is interesting because there is a good mix of attributes -- continuous, nominal with small numbers of values, and nominal with larger numbers of values. There are also a few missing values.

Attribute Information:
- A1: b, a.
- A2: continuous.
- A3: continuous.
- A4: u, y, l, t.
- A5: g, p, gg.
- A6: c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff.
- A7: v, h, bb, j, n, z, dd, ff, o.
- A8: continuous.
- A9: t, f.
- A10: t, f.
- A11: continuous.
- A12: t, f.
- A13: g, p, s.
- A14: continuous.
- A15: continuous.
- A16: +,- (class attribute)

Yes, most of that doesn't mean anything. A16 (the class attribute) is the most interesting, as it separates the 307 approved cases from the 383 rejected cases. The remaining variables have been obfuscated for privacy - a challenge you may have to deal with in your data science career.

Sprint challenges are evaluated based on satisfactory completion of each part. It is suggested you work through it in order, getting each aspect reasonably working, before trying to deeply explore, iterate, or refine any given step. Once you get to the end, if you want to go back and improve things, go for it!

## Part 1 - Load and validate the data

- Load the data as a `pandas` data frame.
- Validate that it has the appropriate number of observations (you can check the raw file, and also read the dataset description from UCI).
- UCI says there should be missing data - check, and if necessary change the data so pandas recognizes it as na
- Make sure that the loaded features are of the types described above (continuous values should be treated as float), and correct as necessary

This is review, but skills that you'll use at the start of any data exploration. Further, you may have to do some investigation to figure out which file to load from - that is part of the puzzle.

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import stats

In [0]:
credit = pd.read_csv('https://www.dropbox.com/s/69nl3gllf1lcyj9/crx.data?dl=1',
                    na_values = ['?'], 
                    header = None,
                    names = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10',
                             'A11', 'A12','A13','A14','A15','A16'])




In [73]:
pd.set_option('display.max_columns', 20)

"""My internet is super spotty right now. I'm working from a local worflow, and 
    I want to make sure that I don't have to import data again in case my internet drops."""

df = credit

print(df.shape)
print()
print(df.isnull().sum())
print()
print(df.describe())
print()
print(df.dtypes)
print()
df.head()

(690, 16)

A1     12
A2     12
A3      0
A4      6
A5      6
A6      9
A7      9
A8      0
A9      0
A10     0
A11     0
A12     0
A13     0
A14    13
A15     0
A16     0
dtype: int64

               A2          A3          A8        A11          A14  \
count  678.000000  690.000000  690.000000  690.00000   677.000000   
mean    31.568171    4.758725    2.223406    2.40000   184.014771   
std     11.957862    4.978163    3.346513    4.86294   173.806768   
min     13.750000    0.000000    0.000000    0.00000     0.000000   
25%     22.602500    1.000000    0.165000    0.00000    75.000000   
50%     28.460000    2.750000    1.000000    0.00000   160.000000   
75%     38.230000    7.207500    2.625000    3.00000   276.000000   
max     80.250000   28.000000   28.500000   67.00000  2000.000000   

                 A15  
count     690.000000  
mean     1017.385507  
std      5210.102598  
min         0.000000  
25%         0.000000  
50%         5.000000  
75%       395.500000  
max    10

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,+


In [74]:
# Everything checks out. I want to change A16 to something more flexible. 

df["A16"].unique()

array(['+', '-'], dtype=object)

In [0]:
#Some minor data cleaning. 
df["A16"] = df["A16"].replace({'+': 1, '-': 0})
df['A11'] = df['A11'].astype(float)
df['A15'] = df['A15'].astype(float)

In [76]:
df.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1.0,f,g,202.0,0.0,1
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6.0,f,g,43.0,560.0,1
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0.0,f,g,280.0,824.0,1
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5.0,t,g,100.0,3.0,1
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0.0,f,s,120.0,0.0,1


## Part 2 - Exploring data, Testing hypotheses

The only thing we really know about this data is that A16 is the class label. Besides that, we have 6 continuous (float) features and 9 categorical features.

Explore the data: you can use whatever approach (tables, utility functions, visualizations) to get an impression of the distributions and relationships of the variables. In general, your goal is to understand how the features are different when grouped by the two class labels (`+` and `-`).

For the 6 continuous features, how are they different when split between the two class labels? Choose two features to run t-tests (again split by class label) - specifically, select one feature that is *extremely* different between the classes, and another feature that is notably less different (though perhaps still "statistically significantly" different). You may have to explore more than two features to do this.

For the categorical features, explore by creating "cross tabs" (aka [contingency tables](https://en.wikipedia.org/wiki/Contingency_table)) between them and the class label, and apply the Chi-squared test to them. [pandas.crosstab](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html) can create contingency tables, and [scipy.stats.chi2_contingency](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html) can calculate the Chi-squared statistic for them.

There are 9 categorical features - as with the t-test, try to find one where the Chi-squared test returns an extreme result (rejecting the null that the data are independent), and one where it is less extreme.

**NOTE** - "less extreme" just means smaller test statistic/larger p-value. Even the least extreme differences may be strongly statistically significant.

Your *main* goal is the hypothesis tests, so don't spend too much time on the exploration/visualization piece. That is just a means to an end - use simple visualizations, such as boxplots or a scatter matrix (both built in to pandas), to get a feel for the overall distribution of the variables.

This is challenging, so manage your time and aim for a baseline of at least running two t-tests and two Chi-squared tests before polishing. And don't forget to answer the questions in part 3, even if your results in this part aren't what you want them to be.

### Continous Feature Analysis

In [0]:
continuous = df[['A2','A3','A8','A11','A14','A15', 'A16']]
categorical = df[['A1','A4','A5','A6','A7','A9','A10','A12','A13','A16']]
approve = continuous.loc[continuous['A16'] == 1]
decline = continuous.loc[continuous['A16'] == 0]

In [78]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
A2,678.0,31.568171,11.957862,13.75,22.6025,28.46,38.2300,80.25
A3,690.0,4.758725,4.978163,0.00,1.0000,2.75,7.2075,28.00
A8,690.0,2.223406,3.346513,0.00,0.1650,1.00,2.6250,28.50
A11,690.0,2.400000,4.862940,0.00,0.0000,0.00,3.0000,67.00
A14,677.0,184.014771,173.806768,0.00,75.0000,160.00,276.0000,2000.00
A15,690.0,1017.385507,5210.102598,0.00,0.0000,5.00,395.5000,100000.00
A16,690.0,0.444928,0.497318,0.00,0.0000,0.00,1.0000,1.00


In [79]:
approve.describe().T

,count,mean,std,min,25%,50%,75%,max
A2,305.0,33.720492,12.809641,13.75,23.17,30.50,41.33,76.75
A3,307.0,5.904951,5.471485,0.00,1.50,4.46,9.52,28.00
A8,307.0,3.427899,4.120792,0.00,0.75,2.00,5.00,28.50
A11,307.0,4.605863,6.320242,0.00,0.00,3.00,7.00,67.00
A14,301.0,164.421927,161.770675,0.00,0.00,120.00,280.00,840.00
A15,307.0,2038.859935,7659.763941,0.00,0.00,221.00,1209.00,100000.00
A16,307.0,1.000000,0.000000,1.00,1.00,1.00,1.00,1.00


In [80]:
decline.describe().T

,count,mean,std,min,25%,50%,75%,max
A2,373.0,29.808231,10.919291,15.17,22.000,27.330,34.83,80.250
A3,383.0,3.839948,4.337662,0.00,0.835,2.210,5.00,26.335
A8,383.0,1.257924,2.120481,0.00,0.125,0.415,1.50,13.875
A11,383.0,0.631854,1.900049,0.00,0.000,0.000,0.00,20.000
A14,376.0,199.699468,181.564835,0.00,100.000,167.500,272.00,2000.000
A15,383.0,198.605744,671.608839,0.00,0.000,1.000,67.00,5552.000
A16,383.0,0.000000,0.000000,0.00,0.000,0.000,0.00,0.000


In [0]:
def dbl_test (column):
    return(stats.ttest_1samp(approve[column], df[column].mean(), nan_policy = 'omit') ,
           stats.ttest_1samp(decline[column], df[column].mean(), nan_policy = 'omit'))

In [82]:
# Not to print my results. 

print('Less Extreme Example: A-2')
print(dbl_test('A2'))
print()
print('More Extreme Example: A-11')
print(dbl_test('A11'))
print()
print('Weird and Fun Example: A-15')
print(dbl_test('A15'))

Less Extreme Example: A-2
(Ttest_1sampResult(statistic=2.934404328473507, pvalue=0.0035964225006290544), Ttest_1sampResult(statistic=-3.112848373560533, pvalue=0.001996279345797444))

More Extreme Example: A-11
(Ttest_1sampResult(statistic=6.115247963474785, pvalue=2.9376851337382315e-09), Ttest_1sampResult(statistic=-18.211793670382843, pvalue=8.54839203623047e-54))

Weird and Fun Example: A-15
(Ttest_1sampResult(statistic=2.336583473592097, pvalue=0.020106005451071305), Ttest_1sampResult(statistic=-23.858881691121024, pvalue=1.1003060833297629e-77))


### Categorical Feature Analysis

In [0]:
def qcross (x):
    """Just to make the crosstabs a little bit quicker"""
    z = pd.crosstab(x,df['A16'],margins = True)
    return z

In [84]:
# The lectures mentioned a smell test. 
# I prefer the term giggle test. Same thing really. 
# Features appear independent. 
qcross(df['A1'])

A16,0,1,All
A1,,,
a,112,98,210
b,262,206,468
All,374,304,678


In [85]:
# Next giggle test. 
# Lets see how it works when I plug in the numebrs. 
# Features appear highly dependent. 
qcross(df['A9'])

A16,0,1,All
A9,,,
f,306,23,329
t,77,284,361
All,383,307,690


In [86]:
# Also as expected. Significantly insignificant. 
# Null Hypothesis NOT rejected. Features appear independent. 
stats.chi2_contingency(qcross(df['A1']))

(0.41143508194591916,
 0.981530018361231,
 4,
 array([[115.84070796,  94.15929204, 210.        ],
        [258.15929204, 209.84070796, 468.        ],
        [374.        , 304.        , 678.        ]]))

In [87]:
# As expected. A9 shows a high degree of significance. 
# Null Hypothesis rejected. Features are not independent. 
stats.chi2_contingency(qcross(df['A9']))

(358.1003264716328,
 3.125328283651331e-76,
 4,
 array([[182.61884058, 146.38115942, 329.        ],
        [200.38115942, 160.61884058, 361.        ],
        [383.        , 307.        , 690.        ]]))

## Part 3 - Analysis and Interpretation

Now that you've looked at the data, answer the following questions:

- Interpret and explain the two t-tests you ran - what do they tell you about the relationships between the continuous features you selected and the class labels?
- Interpret and explain the two Chi-squared tests you ran - what do they tell you about the relationships between the categorical features you selected and the class labels?
- What was the most challenging part of this sprint challenge?

Answer with text, but feel free to intersperse example code/results or refer to it from earlier.

### T Tests

The T-Tests I ran gave me the impression that the Class Labels are fairly dependent on the continuous features. I ran three T-Tests, one highlighting an extreme example and two highlighting less extreme example. The extreme example was the class feature 'A-11' as it produced astronomically small p-values. This indicated that the null hypothesis that these two features can be rejected. 

As a less extreme example, I chose the class feature 'A-2', as it had a less small but still statistically significant p-value. This also indicated that this feature rejects the null hypothesis.

I chose the third example to solidify my next point, that certain class features have a high statistical significance in situations where the individual was declined, but had less apparent (although statistically signficant) impact on whether or not an individual was approved. In both the 'A-11' and 'A-15' have a far higher p-value for those declined than those approved. This is due to a stark difference in Standard Deviation as in both cases, the Standard Deviation of those approved is much larger compared to those declined. 

If I were to intuit why these standard deviations are different, I would state that the features these columns refer to are income related, as those approved could have wildly different incomes while those who are not could have more similar (and lower) incomes reported. 

Results reproduced below:

In [88]:
print('Less Extreme Example: A-2')
print(dbl_test('A2'))
print()
print('More Extreme Example: A-11')
print(dbl_test('A11'))
print()
print('Weird and Fun Example: A-15')
print(dbl_test('A15'))

Less Extreme Example: A-2
(Ttest_1sampResult(statistic=2.934404328473507, pvalue=0.0035964225006290544), Ttest_1sampResult(statistic=-3.112848373560533, pvalue=0.001996279345797444))

More Extreme Example: A-11
(Ttest_1sampResult(statistic=6.115247963474785, pvalue=2.9376851337382315e-09), Ttest_1sampResult(statistic=-18.211793670382843, pvalue=8.54839203623047e-54))

Weird and Fun Example: A-15
(Ttest_1sampResult(statistic=2.336583473592097, pvalue=0.020106005451071305), Ttest_1sampResult(statistic=-23.858881691121024, pvalue=1.1003060833297629e-77))


### $\chi^2$ Tests

The two Categorical features I looked at were **A-1** and **A-9** and they are diametric in how they relate to the Class Feature. **A-1** does not show enough deviation from the null hypothesis. It's p-value is actually quite high, and correlates with the relatively even distribution between categories as associated with the Class Label . On the other hand, **A-9** shows a rather extreme, and noticeable distrubtion between those accepted and those declined. As a result, the $\chi^2$ test clearly rejects the null hypothesis that **A-9** and the Class Label are independent. 

P-Values are shown below:

In [89]:
print('A-1 p-value: ', stats.chi2_contingency(qcross(df['A1']))[1])
print('Null Hypothesis: Not Rejected')
print()
print('A-9 p-value: ', stats.chi2_contingency(qcross(df['A9']))[1])
print('Null Hypothesis: Rejected ')

A-1 p-value:  0.981530018361231
Null Hypothesis: Not Rejected

A-9 p-value:  3.125328283651331e-76
Null Hypothesis: Rejected 


Furthermore, you can see that most features reject the null hypothesis. Only a couple do not reject the null hypothesis and appear to be independent. 

In [90]:
features = categorical.columns.drop('A16').values.tolist()

p_values = []

for i in features:    
  p_values.append(stats.chi2_contingency(qcross(df[i]))[1])
  
categorical_nH = pd.DataFrame({"Features": features, 
                               "P-Values": p_values, 
                               "Null Hypothesis Rejected": False})

categorical_nH.loc[categorical_nH['P-Values'] < .05, "Null Hypothesis Rejected"] = True  

categorical_nH

,Features,Null Hypothesis Rejected,P-Values
0,A1,False,9.815300e-01
1,A4,True,2.013603e-04
2,A5,True,2.013603e-04
3,A6,True,9.446934e-10
4,A7,True,4.097262e-04
5,A9,True,3.125328e-76
6,A10,True,2.485585e-30
7,A12,False,9.525455e-01
8,A13,False,1.630875e-01


### Most Challenging Part: 

Sifting through the data to find the most likely candidates for the Statistical Tests. I created a few functions to do it more quickly, but it was the part the required the most care. 